# Simulation d'un revenu de base financé par la CSG

### Test de prise en compte du revenu net au lieu du revenu imposable dans le revenu disponible

In [31]:
# Hypothèses retenues pour la réforme simulée :

#  Maintien de l'IRPP
#- Hausse de la CSG imposable au taux de 23.333% (+ 5.1% le taux actuel)
#- Suppression de la CSG dductible
#- Mise en place d'un revenu de base adulte et d'un revenu de base enfant : attention, le montant de la csg déductible 
#  complémentaire est bien évidemment fonction du montant du revenu de base à financer, et son calcul se fait sur la base 
#  du besoin d'équilibrage des finances publiques qui est à calculer en dehors de ce notebook.
#- Intégration du revenu de base, adulte et enfants, au revenu disponible
#- Suppression du RSA, des allocations familiales, du complément familial, de l'ars et de la prime d'activités

# Le notebook est organisé de la manière suivante :
# 1. Système socio-fiscal : appel de tax_benefit_system et initialisation des principales variables
# 2. Réforme permettant de créer les class des nouvelles variables et d'en modifier certaines existant déjà dans tax_benefit_system
# 3. Tests permettant, sur la base de montants de salaires fixes, de tester les variables dans la situation actuelle et dans la situation avec réforme



In [32]:
from datetime import date
import json
import openfisca_france
from openfisca_france.model.base import * 
import csv
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
from numpy import round

# to debug / trace
from openfisca_core import periods,tools
from openfisca_core.reforms import Reform

## 1. Système socio-fiscal

In [33]:
tax_benefit_system = openfisca_france.FranceTaxBenefitSystem()

In [34]:
# Initialisation des varialbes relatives au montant du revenu de base et au taux de la csg déductible complémentaire

rdb_adulte = 500
rdb_enfant_0_a_18 = 250
csg_deduc_complem = 0.28433

In [35]:
#Initialisation de la fourchette des salaires à simuler et du calcul du nombre de pas à itérer

min_salaire_de_base = 0
max_salaire_de_base = 100000
pas =450*12
nb_palier = max_salaire_de_base / pas
nb_palier

18

In [36]:
period = 2016

In [37]:
#- Modification du montant du paramètre csg.activite.deductible.taux destiné à financer le revenu de base

def modify_legislation_json(reference_legislation_json_copy):
    for value_json in reference_legislation_json_copy['children']['prelevements_sociaux']['children']['contributions']['children']['csg']['children']['activite']['children']['imposable']['children']['taux']['values']:
        value_json['value'] = csg_deduc_complem
    for value_json in reference_legislation_json_copy['children']['prelevements_sociaux']['children']['contributions']['children']['csg']['children']['activite']['children']['deductible']['children']['taux']['values']:
        value_json['value'] = 0
        
    return reference_legislation_json_copy

## 2. Réforme : Revenu de base par CSG

In [38]:
#- Mise en place d'un revenu de base adulte 

class rdb(Variable):
    column = FloatCol
    entity = Individu
    label = u"Revenu de base"
    definition_period = MONTH
       
    def function(individu, period, legislation):
        period = period.start.offset('first-of', 'month').period('month')
        age = individu('age', period) 
        return ((age >= 18) * rdb_adulte  + not_(age >= 18) * 0)

In [39]:
# Mise en place d'un revenu de base enfant

class rdb_enf(Variable):
    column = FloatCol
    entity = Individu
    label = u"Revenu de base enfant"
    definition_period = MONTH

    def function(individu, period, legislation):
        period = period.start.offset('first-of', 'month').period('month')
        age = individu('age', period)
        return ((age < 14) * rdb_enfant_0_a_18 + not_(age < 14) * 250) * (age <= 18)

In [40]:
# Ajout du revenu de base adulte et enfant au revenu disponible
# Ajout de "csg_imposable_salaire + csg_imposable_retraite + crds" pour prise en compte des revenus nets et non plus des revenus imposables

class revenu_disponible(Variable):
    column = FloatCol
    entity = Menage
    label = u"Revenu disponible du ménage"
    url = "http://fr.wikipedia.org/wiki/Revenu_disponible"
    definition_period = YEAR

    def function(self, simulation, period):
        revenus_du_travail_holder = simulation.compute('revenus_du_travail', period)
        pen_holder = simulation.compute('pensions', period)
        rev_cap_holder = simulation.compute('revenus_du_capital', period)
        prestations_sociales_holder = simulation.compute('prestations_sociales', period)
        ppe_holder = simulation.compute('ppe', period)
        impots_directs = simulation.calculate('impots_directs', period)
        rdb_holder = simulation.calculate_add('rdb', period)
        rdb_enf_holder = simulation.calculate_add('rdb_enf', period)
        csg_imposable_salaire_holder = simulation.calculate_add('csg_imposable_salaire', period)
        csg_imposable_retraite_holder = simulation.calculate_add('csg_imposable_retraite', period)
        crds_holder = simulation.calculate_add('crds', period)
        
        pen = self.sum_by_entity(pen_holder)
        ppe = self.cast_from_entity_to_role(ppe_holder, role = VOUS)
        ppe = self.sum_by_entity(ppe)
        prestations_sociales = self.cast_from_entity_to_role(prestations_sociales_holder, role = CHEF)
        prestations_sociales = self.sum_by_entity(prestations_sociales)
        rev_cap = self.sum_by_entity(rev_cap_holder)
        revenus_du_travail = self.sum_by_entity(revenus_du_travail_holder)
        rdb = self.sum_by_entity(rdb_holder)
        rdb_enf = self.sum_by_entity(rdb_enf_holder)
        csg_imposable_salaire = self.sum_by_entity(csg_imposable_salaire_holder)
        csg_imposable_retraite = self.sum_by_entity(csg_imposable_retraite_holder)
        crds = self.sum_by_entity(crds_holder)
    
        return ( 
            revenus_du_travail + pen + rev_cap + prestations_sociales + ppe + impots_directs + 
            rdb + rdb_enf +
            csg_imposable_salaire + csg_imposable_retraite + crds
            )

In [41]:
class ReformeRevenuDeBaseparCSG(Reform):
    name = u"Réforme Revenu de base par CSG"

    def apply(self):
        # Suppression de variables
        self.neutralize_variable('af')
        self.neutralize_variable('cf')
        self.neutralize_variable('ars')
        self.neutralize_variable('rsa_socle')
        self.neutralize_variable('rsa')
        self.neutralize_variable('ppa')
     
        # Mise à jour de variables
        for variable in [rdb, rdb_enf, revenu_disponible]:
            self.update_variable(variable)
            
        # Mise à jour des paramètres de la législation
        self.modify_legislation_json(modifier_function = modify_legislation_json)

## 3. Tests

In [42]:
reform = ReformeRevenuDeBaseparCSG(tax_benefit_system)

In [61]:
# Initialisation du montant des salaires des 2 parents

parent1_salaire_de_base = 50000
parent2_salaire_de_base = 30000


In [62]:
scenario_ref_individu_seul = tax_benefit_system.new_scenario().init_single_entity(
    period = 2016,
    parent1 = dict(
        date_naissance = date(1980, 1, 1),
        salaire_de_base = parent1_salaire_de_base,
        statut_marital = u'Célibataire',
        ),
    parent2 = dict(
        date_naissance = date(1980, 1, 1),
        salaire_de_base = parent2_salaire_de_base,
        statut_marital = u'Célibataire',
        ),
    foyer_fiscal = dict(
        caseT = True,        
        ),
    enfants = [
        dict(date_naissance = date(2005, 1, 1)),
        dict(date_naissance = date(2011, 1, 1)),
     ],
    menage = dict(
         loyer = 0,
         statut_occupation_logement = 4,
         zone_apl = 1,
         ),
        
    )

In [63]:
simulation_ref_individu_seul = scenario_ref_individu_seul.new_simulation(debug = True)

In [64]:
scenario_rdb_individu_seul = reform.new_scenario().init_single_entity(
    period = 2016,
    parent1 = dict(
        date_naissance = date(1980, 1, 1),
        salaire_de_base = parent1_salaire_de_base,
        statut_marital = u'Célibataire',
        ),
    parent2 = dict(
        date_naissance = date(1980, 1, 1),
        salaire_de_base = parent2_salaire_de_base,
        statut_marital = u'Célibataire',
        ),
    foyer_fiscal = dict(
        caseT = True,
                ),
    enfants = [
        dict(date_naissance = date(2005, 1, 1)),
        dict(date_naissance = date(2011, 1, 1)),
     ],
    menage = dict(
         loyer = 0,
         statut_occupation_logement = 4,
         zone_apl = 1,
         ),
    
    )

In [65]:
simulation_rdb_individu_seul = scenario_rdb_individu_seul.new_simulation(debug = True)

### Calculs de référence des variables les plus pertinentes dans le système actuel


In [66]:
simulation_ref_individu_seul.calculate_add('irpp', period)/12

array([-564.1762085], dtype=float32)

In [67]:
simulation_ref_individu_seul.calculate('revenu_disponible', period)

array([ 60329.65234375], dtype=float32)

In [69]:
simulation_ref_individu_seul.calculate_add('csg_imposable_salaire', period)

array([-1178.99902344,  -707.40008545,    -0.        ,    -0.        ], dtype=float32)

In [70]:
simulation_ref_individu_seul.calculate_add('csg_deductible_salaire', period)

array([-2505.37231445, -1503.22509766,    -0.        ,    -0.        ], dtype=float32)

In [71]:
simulation_ref_individu_seul.calculate_add('csg', period)

array([-3684.37133789, -2210.62524414,     0.        ,     0.        ], dtype=float32)

### Calculs de référence des variables les plus pertinentes dans le système réformé avec un revenu de base

In [72]:
simulation_rdb_individu_seul.calculate('irpp', period)/12

array([-654.36621094], dtype=float32)

In [73]:
simulation_rdb_individu_seul.calculate('revenu_disponible', period)

array([ 56215.16015625], dtype=float32)

In [74]:
simulation_rdb_individu_seul.calculate_add('rdb', period)/12

array([ 500.,  500.,    0.,    0.], dtype=float32)

In [75]:
simulation_rdb_individu_seul.calculate_add('rdb_enf', period)/12

array([   0.,    0.,  250.,  250.], dtype=float32)

In [76]:
simulation_rdb_individu_seul.calculate_add('csg_imposable_salaire', period)

array([-13967.69824219,  -8380.62792969,     -0.        ,     -0.        ], dtype=float32)

In [77]:
simulation_rdb_individu_seul.calculate_add('csg_deductible_salaire', period)

array([-0., -0., -0., -0.], dtype=float32)

In [78]:
simulation_rdb_individu_seul.calculate_add('csg', period)

array([-13967.69824219,  -8380.62792969,      0.        ,      0.        ], dtype=float32)